<a href="https://colab.research.google.com/github/chrisbrighouse/Public/blob/main/Personas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip 
!pip install --no-cache-dir openai 
!pip install glob2 

#!pipenv install openai 
#!pip install -q openai 

# Authenticate to Google cloud drive
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive


In [ ]:
from prompt_toolkit.eventloop import context
#@title
import openai
import os
import requests
import re
from IPython.display import Audio
from google.colab import drive

#API Keys
elapikey = open_file (file_folder_path + "API Keys/elevenlabskey.txt").strip()
openai.api_key = open_file (file_folder_path + "API Keys/openaikey.txt").strip()

#Elevenlabs Voices
preview = "21m00Tcm4TlvDq8ikWAM"
Ellie = "MF3mGyEYCl7XYWbV9V6O"
Domi = "AZnzlk1XvdvUeBnXmlld"
Bella = "EXAVITQu4vr4xnSDxMaL"
Antoni = "ErXwobaYiN019PkySvjV"
Josh = "TxGEqnHWrfWFTfGW9XjX"
Arnold = "VR6AewLTigWG4xSOukaG"
Adam = "pNInz6obpgDQGcFmaJgB"
Sam = "yoZ06aMxZJJ28mfd3POQ"

# Define the path to the Google Drive top level
file_folder_path = '/gdrive/MyDrive/'

def open_file(filepath):
  print (f"opening file: {filepath}")
  with open(filepath, 'r', encoding='utf-8') as infile:
    return infile.read()

def output_file_replace (filepath):
  print (f"output file: {filepath}")
  with open(filepath, 'wb', )as f:
    f.write(response.content)
    return f

#function to reset or update the chat context in a file
def chat_context(context, bAppend):
  output_file = f"{file_folder_path}Output/context.txt"
  print(output_file)
  if bAppend:
    with open (output_file, 'a', )as f:
      f.write (context)
  else:
    with open (output_file, 'w', )as f:
      f.write(context)

  with open (output_file, "r") as f:    
    context = f.read()

  return context


def run_openai_prompt (prompt, bAddToContext, bNoResponse):
  #set up the OpenAI call
  model_engine = "text-davinci-003"
  #model_engine = "text-davinci-002"
  #model_engine = "davinci"
  if bNoResponse:
    n=0
  else:
    n=1

  context=chat_context (prompt, bAddToContext)
  print (f"Context before call: {context}")

  try:
    completion = openai.Completion.create( 
      engine=model_engine, 
      prompt=prompt,
      max_tokens=100, 
      n=n, 
      stop=None, 
      temperature=0.8,
      #context=context,
    )

    print("completion: \n", completion)
    
    #send to OpenAI
    response = completion.choices[0].text.strip()

    #save context 
    context=chat_context (response, True)
    print (f"Context after call: {context}")

    print(response)
    return response

  except Exception as e:
    print(f"Error: {e}")

#Main routine
if __name__ == '__main__':
  
  #This name will be used to find the prompt and generate the output file
  role_name="MrGrump"

  #API Keys
  elapikey = open_file (file_folder_path + "API Keys/elevenlabskey.txt").strip()
  openai.api_key = open_file (file_folder_path + "API Keys/openaikey.txt").strip()

  # Define the name of the text file you want to read
  file_name = f"Personas/{role_name}.txt"

  #get the persona input file
  prompt = open_file (file_folder_path + file_name)
  
  print (f"Prompt: {prompt}")

  #start with a new conversation
  response = run_openai_prompt (prompt, False, False)

  #add to the conversation in context
  response = run_openai_prompt (f"{response}\nHave you got a smartphone, what do you think of it?\n", True, False)

  #input_text = prompt + " " + response
  input_text = response
  print ("Elevenlabs input: ", input_text)

  voice_id = Adam
  elevenlabs_tts_url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
  print (elevenlabs_tts_url)
  output_file = f"{file_folder_path}Output/{role_name}.wav"
  print(output_file)

  #Request payload
  headers = {
      "xi-api-key": elapikey,
      "accept": "audio/mpeg",
      "Content-Type": "application/json"
  }

  data = {
      "text": input_text,
      "lang": "en_US",
  }

  #Send request to Elevenlabs TTS API
  response = requests.post(elevenlabs_tts_url, headers=headers, json=data)

  # Check response status
  if response.status_code == 200:
    #Write audio to file
    output_file_replace (output_file)
   
    #Display audio
    display(Audio(filename=output_file))
  else:
    print(f"Error: {response.status_code}")

